In [2]:
from __future__ import annotations

from pathlib import Path
import rich

import mattertune.configs as MC
from mattertune import MatterTuner


def hparams():
    hparams = MC.MatterTunerConfig.draft()

    # Model hparams
    hparams.model = MC.ORBBackboneConfig.draft()
    hparams.model.pretrained_model = "orb-v2"
    hparams.model.ignore_gpu_batch_transform_error = True
    hparams.model.optimizer = MC.AdamWConfig(lr=1.0e-4)

    hparams.model.properties = []
    energy = MC.EnergyPropertyConfig(loss=MC.MAELossConfig())
    hparams.model.properties.append(energy)
    forces = MC.ForcesPropertyConfig(loss=MC.MAELossConfig(), conservative=False)
    hparams.model.properties.append(forces)
    stresses = MC.StressesPropertyConfig(loss=MC.MAELossConfig(), conservative=False)
    hparams.model.properties.append(stresses)

    # Data hparams
    hparams.data = MC.ManualSplitDataModuleConfig.draft()
    hparams.data.train = MC.OMAT24DatasetConfig.draft()
    hparams.data.train.src = Path("/mnt/datasets/salex/val/")
    hparams.data.batch_size = 1
    hparams.data.num_workers = 8

    # Trainer hparams
    hparams.trainer.additional_trainer_kwargs = {
        "fast_dev_run": True,
        # "devices": [0],
        "accelerator": "cpu",
    }

    hparams = hparams.finalize()
    rich.print(hparams)
    return hparams


hp = hparams()
tune_output = MatterTuner(hp).tune()
model, trainer = tune_output.model, tune_output.trainer

MatterTunerConfig(
    data=ManualSplitDataModuleConfig(
        batch_size=1,
        num_workers=8,
        pin_memory=True,
        train=OMAT24DatasetConfig(type='omat24', src=PosixPath('/mnt/datasets/salex/val')),
        validation=None
    ),
    model=ORBBackboneConfig(
        reset_backbone=False,
        freeze_backbone=False,
        reset_output_heads=True,
        use_pretrained_normalizers=False,
        properties=[
            EnergyPropertyConfig(
                name='energy',
                dtype='float',
                loss=MAELossConfig(name='mae', reduction='mean'),
                loss_coefficient=1.0,
                type='energy'
            ),
            ForcesPropertyConfig(
                name='forces',
                dtype='float',
                loss=MAELossConfig(name='mae', reduction='mean'),
                loss_coefficient=1.0,
                type='forces',
                conservative=False
            ),
            StressesPropertyConfig(
                name='stresses',
                dtype='float',
                loss=MAELossConfig(name='mae', reduction='mean'),
                loss_coefficient=1.0,
                type='stresses',
                conservative=False
            )
        ],
        optimizer=AdamWConfig(
            per_parameter_hparams=None,
            name='AdamW',
            lr=0.0001,
            eps=1e-08,
            betas=(0.9, 0.999),
            weight_decay=0.01,
            amsgrad=False
        ),
        lr_scheduler=None,
        ignore_gpu_batch_transform_error=True,
        normalizers={},
        name='orb',
        pretrained_model='orb-v2',
        system=ORBSystemConfig(radius=6.0, max_num_neighbors=120)
    ),
    trainer=TrainerConfig(
        accelerator='auto',
        strategy='auto',
        num_nodes=1,
        devices='auto',
        precision='32-true',
        deterministic=None,
        max_epochs=None,
        min_epochs=None,
        max_steps=-1,
        min_steps=None,
        max_time=None,
        val_check_interval=None,
        check_val_every_n_epoch=1,
        log_every_n_steps=None,
        gradient_clip_val=None,
        gradient_clip_algorithm=None,
        checkpoint=None,
        early_stopping=None,
        ema=None,
        loggers='default',
        additional_trainer_kwargs={'fast_dev_run': True, 'accelerator': 'cpu'}
    ),
    recipes=[]
)

/net/csefiles/coc-fung-cluster/lingyu/miniconda3/envs/orbv3-tune/lib/python3.10/site-packages/orb_models/utils.py:30: UserWarning: Setting global torch default dtype to torch.float32.
  warnings.warn(f"Setting global torch default dtype to {torch_dtype}.")
You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/net/csefiles/coc-fung-cluster/lingyu/miniconda3/envs/orbv3-tune/lib/python3.10/site-packages/lightning/pytorch/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.


ImportError: The `fairchem` package is not installed. Please install it by running `pip install fairchem`.

In [ ]:
property_predictor = model.property_predictor()
print(property_predictor)

calculator = model.ase_calculator()
print(calculator)

In [ ]:
import ase

# Create a test periodic system
atoms = ase.Atoms(
    "H2O", positions=[[0, 0, 0], [0, 0, 1], [0, 1, 0]], cell=[10, 10, 10], pbc=True
)
print(atoms)

print(property_predictor.predict([atoms], model.hparams.properties))

INFO: GPU available: True (cuda), used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.
INFO:lightning.pytorch.utilities.rank_zero:Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.


Atoms(symbols='H2O', pbc=True, cell=[10.0, 10.0, 10.0])


/opt/conda/envs/jmp-peft/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.


Predicting: |          | 0/? [00:00<?, ?it/s]

[{'energy': tensor[1, 1] [[0.044]], 'forces': tensor[3, 3] n=9 x∈[-0.183, 0.094] μ=1.242e-09 σ=0.084 [[-0.056, 0.028, 0.089], [0.016, -0.021, 0.094], [0.040, -0.008, -0.183]], 'stresses': tensor[1, 3, 3] n=9 x∈[-0.135, 0.108] μ=-0.055 σ=0.090 [[[-0.085, 0.037, -0.135], [0.037, 0.108, -0.105], [-0.135, -0.105, -0.107]]]}]


In [5]:
import ase

# Create a test periodic system
atoms = ase.Atoms(
    "H2O", positions=[[0, 0, 0], [0, 0, 1], [0, 1, 0]], cell=[10, 10, 10], pbc=True
)
print(atoms)

# Set the calculator
atoms.calc = calculator

# Calculate the energy
energy = atoms.get_potential_energy()
print(energy)

INFO: GPU available: True (cuda), used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.
INFO:lightning.pytorch.utilities.rank_zero:Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.


Atoms(symbols='H2O', pbc=True, cell=[10.0, 10.0, 10.0])


Predicting: |          | 0/? [00:00<?, ?it/s]

-0.06458454579114914


In [ ]:
property_predictor = model.property_predictor()
atoms_1 = ase.Atoms(
    "H2O", positions=[[0, 0, 0], [0, 0, 1], [0, 1, 0]], cell=[10, 10, 10], pbc=True
)
atoms_2 = ase.Atoms(
    "H2O", positions=[[0, 0, 0], [0, 0, 1], [0, 1, 0]], cell=[10, 10, 10], pbc=True
)
atoms = [atoms_1, atoms_2]
predictions = property_predictor.predict(atoms, ["energy", "forces"])
print("ase.Atoms 1 energy:", predictions[0]["energy"])
print("ase.Atoms 1 forces:", predictions[0]["forces"])
print("ase.Atoms 2 energy:", predictions[1]["energy"])
print("ase.Atoms 2 forces:", predictions[1]["forces"])

INFO: Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
INFO:lightning.pytorch.utilities.rank_zero:Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU av

DEBUG:fsspec.local:open file: /workspaces/MatterTune/examples/lightning_logs/version_22/hparams.yaml


Predicting: |          | 0/? [00:00<?, ?it/s]

ase.Atoms 1 energy: tensor[1, 1] [[-0.065]]
ase.Atoms 1 forces: tensor[3, 3] n=9 x∈[-0.167, 0.117] μ=8.278e-10 σ=0.089 [[-0.067, 0.050, -0.167], [-0.016, 0.017, 0.117], [0.083, -0.068, 0.050]]
ase.Atoms 2 energy: tensor[1, 1] [[-0.065]]
ase.Atoms 2 forces: tensor[3, 3] n=9 x∈[-0.167, 0.117] μ=8.278e-10 σ=0.089 [[-0.067, 0.050, -0.167], [-0.016, 0.017, 0.117], [0.083, -0.068, 0.050]]
